<a href="https://colab.research.google.com/github/jvmorva/Google-Gemini-Agentes/blob/main/Gemini_Agentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai==1.15.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.3/171.3 kB 4.4 MB/s eta 0:00:00


In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é o próximo evento da AWS em Curitiba ###############################################
response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é o próximo evento da AWS em Curitiba?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 O próximo evento da AWS em Curitiba é o **AWS Community Day Curitiba 2024**, que acontecerá no dia **22 de junho de 2024**.

Você pode encontrar mais informações e se inscrever no evento através do seguinte link:

*   [https://www.sympla.com.br/evento/aws-community-day-curitiba-2024/2449029](https://www.sympla.com.br/evento/aws-community-day-curitiba-2024/2449029)

Recomendo verificar o site oficial do evento para obter os detalhes mais recentes, como a programação completa, palestrantes e local.

In [ ]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é o próximo evento da AWS em Curitiba?',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 O próximo evento da AWS em Curitiba é o AWS Curitiba User Group, que acontecerá no dia 3 de junho de 2025, às 19:00 BRT, no MadeiraMadeira. O tema do encontro será "Agentes de AI na Prática", com a participação de Marcelo Palladino, Senior Developer Advocate da AWS. As vagas são limitadas e é necessário informar o CPF na inscrição para garantir a entrada.

In [ ]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['eventos aws curitiba', 'próximo evento aws curitiba']
Páginas utilizadas na resposta: meetup.com, meetup.com



In [ ]:
# Instalar Framework de agentes do Google
!pip install -q google-adk==1.0.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
import uuid

async def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()

    # Gere um session_id único
    session_id = str(uuid.uuid4())

    # Cria a sessão com esse ID
    await session_service.create_session(
        app_name=agent.name,
        user_id="user1",
        session_id=session_id
    )

    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)

    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""

    async for event in runner.run_async(user_id="user1", new_message=content, session_id=session_id):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text + "\n"

    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):

    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do google (google_search)
        para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja tão relevante assim
        e pode ser substituído por outro que tenha mais.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
        """,
        description="Agente que busca informações no Google",
        tools=[google_search]
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    lancamentos = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos

In [ ]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de
        lançamentos mais recentes e relevantes buscador, você deve:
        usar a ferramenta de busca do Google (google_search) para criar um plano sobre
        quais são os pontos mais relevantes que poderíamos abordar em um post sobre
        cada um deles. Você também pode usar o (google_search) para encontrar mais
        informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas
        e retornar esse tema, seus pontos mais relevantes, e um plano com os assuntos
        a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [ ]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Linkedin sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Linkedin"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [ ]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Linkedin.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Linkedin abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

async def run_agent_system(topico):
    print("🚀 Iniciando o Sistema de Criação de Posts para Linkedin com 4 Agentes 🚀")

    if not topico:
        print("Você esqueceu de digitar o tópico!")
    else:
        print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

        # Chame as funções dos agentes com await
        # Adiciona session_service como argumento para agente_buscador
        lancamentos_buscados = await agente_buscador(topico, data_de_hoje)
        print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
        display(to_markdown(lancamentos_buscados))
        print("--------------------------------------------------------------")

        plano_de_post = await agente_planejador(topico, lancamentos_buscados)
        print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
        display(to_markdown(plano_de_post))
        print("--------------------------------------------------------------")

        rascunho_de_post = await agente_redator(topico, plano_de_post)
        print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
        display(to_markdown(rascunho_de_post))
        print("--------------------------------------------------------------")

        post_final = await agente_revisor(topico, rascunho_de_post)
        print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
        display(to_markdown(post_final))
        print("--------------------------------------------------------------")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Execute a função assíncrona principal diretamente com await em ambientes como Colab/Jupyter
await run_agent_system(topico)

❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: AWS
🚀 Iniciando o Sistema de Criação de Posts para Linkedin com 4 Agentes 🚀
Maravilha! Vamos então criar o post sobre novidades em AWS

--- 📝 Resultado do Agente 1 (Buscador) ---



> Para encontrar os lançamentos mais relevantes da AWS no último mês, vou realizar algumas pesquisas no Google.
> 
> 
> Com base nas minhas pesquisas, aqui estão alguns dos lançamentos e anúncios mais relevantes da AWS no último mês (Maio de 2025):
> 
> 1.  **Investimento Massivo no Chile:** A AWS anunciou um investimento de US$4 bilhões em 15 anos para a construção de três data centers em Santiago, Chile. Esse investimento visa fornecer serviços de nuvem não apenas para o mercado doméstico, mas para toda a América Latina. A nova região da AWS na América do Sul (Chile) terá três zonas de disponibilidade, juntando-se às 114 zonas e 36 regiões que a AWS já opera globalmente.
> 2.  **Amazon ECS com Rollbacks Simplificados:** O Amazon Elastic Container Service (ECS) introduziu um novo recurso que permite reverter facilmente um serviço do Amazon ECS para um estado seguro anterior em caso de falha na implantação. Isso simplifica a recuperação de falhas e aumenta a confiabilidade das implantações.
> 3.  **Amazon Aurora com Suporte para PostgreSQL 17:** O Amazon Aurora agora suporta a versão principal 17 do PostgreSQL (17.4). Esta versão inclui melhorias e correções de bugs da comunidade PostgreSQL, garantindo que os usuários do Aurora possam aproveitar as últimas atualizações e recursos do PostgreSQL.
> 4.  **AWS e IBM Colaboram em IA:** A AWS e a IBM continuam a colaborar para oferecer novos recursos de IA generativa. A integração planejada entre o Amazon Q Index e o IBM watsonx Orchestrate visa facilitar a criação, implantação e gerenciamento de agentes e assistentes de IA. O IBM watsonx Agents e watsonx Orchestrate estão agora disponíveis como SaaS e no AWS Marketplace em várias regiões.
> 5.  **Amazon RDS for Oracle:** O Amazon RDS for Oracle agora suporta o gerenciamento de credenciais com o AWS Secrets Manager para bancos de dados que adotam a arquitetura multitenant da Oracle. Isso permite que os clientes automatizem a rotação de senhas, utilizem o AWS Identity and Access Management (IAM) para controle de acesso e criptografem credenciais usando o AWS Key Management Service (KMS).
> 6.  **Amazon EC2 High Memory Instances:** As instâncias Amazon EC2 High Memory U-1 com 18TB de memória (u-18tb1.112xlarge) estão agora disponíveis na região US East (Ohio). Os clientes podem começar a usar essas novas instâncias com opções de compra On Demand e Savings Plan.
> 7.  **AWS Entity Resolution:** O AWS Entity Resolution está agora disponível nas regiões AWS Canada Central e Africa Cape Town, oferecendo às organizações a capacidade de corresponder e vincular registros relacionados armazenados em várias aplicações.
> 8.  **Novidades de Segurança e Governança:** As atualizações incluem o IAM Access Analyzer com política gerada automaticamente para S3, Centralized Logging na Control Tower, suporte ao Verified Access no CloudTrail Lake e melhorias no Config Rules.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Planejador) ---



> Okay, com base nos lançamentos da AWS de Maio de 2025, vou criar um plano de conteúdo para redes sociais, focando em cada um dos tópicos e identificando os pontos mais relevantes para um post.
> 
> **1. Investimento Massivo no Chile:**
> 
> *   **Pontos Relevantes:**
>     *   Investimento de US$4 bilhões em data centers.
>     *   Construção de três data centers em Santiago.
>     *   Expansão da infraestrutura de nuvem na América Latina.
>     *   Geração de empregos e desenvolvimento tecnológico na região.
>     *   Aumento da capacidade de serviços de nuvem para empresas locais e internacionais.
> 
> *   **Plano de Post:**
>     *   Manchete chamativa: "AWS Investe US$4 Bilhões no Chile: Um Marco para a Nuvem na América Latina!"
>     *   Explicar o tamanho do investimento e o número de data centers.
>     *   Detalhar os benefícios para empresas no Chile e na América Latina.
>     *   Mencionar as novas zonas de disponibilidade e sua importância.
>     *   Incluir uma imagem/vídeo mostrando o impacto do investimento (ex: mapa da região, visualização de data center).
>     *   Call to action: "Saiba como sua empresa pode se beneficiar da infraestrutura AWS na América Latina."
> 
> 
> **2. Amazon ECS com Rollbacks Simplificados:**
> 
> *   **Pontos Relevantes:**
>     *   Novo recurso de rollback para serviços do Amazon ECS.
>     *   Recuperação mais rápida de falhas em implantações.
>     *   Aumento da confiabilidade e estabilidade dos serviços.
>     *   Redução do tempo de inatividade em caso de problemas.
> 
> *   **Plano de Post:**
>     *   Título: "Amazon ECS Facilitando a Vida dos Desenvolvedores: Rollbacks Simplificados!"
>     *   Explicar o que é o Amazon ECS e sua importância.
>     *   Detalhar o novo recurso de rollback e como ele funciona.
>     *   Exemplos de cenários onde o rollback é útil.
>     *   Comparar com a complexidade de rollbacks manuais.
>     *   Incluir um gif/vídeo demonstrando o processo de rollback.
>     *   Call to action: "Teste o novo recurso de rollback no seu próximo deployment no ECS!"
> 
> **3. Amazon Aurora com Suporte para PostgreSQL 17:**
> 
> *   **Pontos Relevantes:**
>     *   Aurora agora suporta PostgreSQL 17.4.
>     *   Acesso às últimas melhorias e correções do PostgreSQL.
>     *   Desempenho aprimorado e novas funcionalidades.
>     *   Compatibilidade com aplicações existentes.
> 
> *   **Plano de Post:**
>     *   Título: "Amazon Aurora + PostgreSQL 17: A Combinação Perfeita para seu Banco de Dados!"
>     *   Explicar o que é o Amazon Aurora e seus benefícios.
>     *   Destacar as principais novidades do PostgreSQL 17.4.
>     *   Focar em como essas novidades beneficiam os usuários do Aurora.
>     *   Facilidade de upgrade para a nova versão.
>     *   Incluir um gráfico comparando o desempenho do Aurora com e sem PostgreSQL 17.
>     *   Call to action: "Atualize seu banco de dados Aurora para o PostgreSQL 17 e aproveite os novos recursos!"
> 
> **4. AWS e IBM Colaboram em IA:**
> 
> *   **Pontos Relevantes:**
>     *   Parceria entre AWS e IBM para oferecer soluções de IA generativa.
>     *   Integração entre Amazon Q Index e IBM watsonx Orchestrate.
>     *   Facilidade na criação, implantação e gerenciamento de agentes e assistentes de IA.
>     *   Disponibilidade do IBM watsonx Agents e Orchestrate no AWS Marketplace.
> 
> *   **Plano de Post:**
>     *   Título: "AWS e IBM Unem Forças para Revolucionar a IA Generativa!"
>     *   Explicar a importância da IA generativa e suas aplicações.
>     *   Detalhar a parceria entre AWS e IBM e seus objetivos.
>     *   Explicar como a integração entre Amazon Q Index e IBM watsonx Orchestrate funciona.
>     *   Benefícios para desenvolvedores e empresas que usam IA.
>     *   Incluir um diagrama da arquitetura da solução integrada.
>     *   Call to action: "Descubra como a IA generativa da AWS e IBM pode transformar seus negócios!"
> 
> 
> **5. Amazon RDS for Oracle:**
> 
> *   **Pontos Relevantes:**
>     *   Suporte ao gerenciamento de credenciais com o AWS Secrets Manager.
>     *   Automatização da rotação de senhas.
>     *   Uso do AWS IAM para controle de acesso.
>     *   Criptografia de credenciais com o AWS KMS.
>     *   Maior segurança e conformidade para bancos de dados Oracle.
> 
> *   **Plano de Post:**
>     *   Título: "Amazon RDS for Oracle Mais Seguro do que Nunca com AWS Secrets Manager!"
>     *   Explicar a importância da segurança em bancos de dados.
>     *   Detalhar como o AWS Secrets Manager protege as credenciais do Oracle.
>     *   Benefícios da automatização da rotação de senhas.
>     *   Cumprimento de normas de segurança e conformidade.
>     *   Incluir um diagrama mostrando a integração entre RDS, Secrets Manager, IAM e KMS.
>     *   Call to action: "Proteja seus bancos de dados Oracle com o AWS Secrets Manager e o Amazon RDS!"
> 
> **6. Amazon EC2 High Memory Instances:**
> 
> *   **Pontos Relevantes:**
>     *   Novas instâncias EC2 High Memory U-1 com 18TB de memória.
>     *   Disponibilidade na região US East (Ohio).
>     *   Ideal para cargas de trabalho que exigem alta capacidade de memória.
>     *   Opções de compra On Demand e Savings Plan.
> 
> *   **Plano de Post:**
>     *   Título: "Amazon EC2 High Memory: Potência Máxima para Suas Aplicações!"
>     *   Explicar o que são as instâncias EC2 High Memory e para que servem.
>     *   Detalhar as especificações das novas instâncias U-1 com 18TB de memória.
>     *   Casos de uso ideais: SAP HANA, bancos de dados in-memory, etc.
>     *   Opções de compra e como economizar com Savings Plan.
>     *   Incluir um gráfico comparando o desempenho das instâncias High Memory com outras instâncias EC2.
>     *   Call to action: "Escalone suas aplicações com as instâncias EC2 High Memory e veja a diferença!"
> 
> **7. AWS Entity Resolution:**
> 
> *   **Pontos Relevantes:**
>     *   Disponibilidade nas regiões AWS Canada Central e Africa Cape Town.
>     *   Capacidade de corresponder e vincular registros relacionados em várias aplicações.
>     *   Melhora a qualidade dos dados e a visão unificada dos clientes.
> 
> *   **Plano de Post:**
>     *   Título: "AWS Entity Resolution Chega a Novas Regiões: Unifique Seus Dados e Conheça Seus Clientes!"
>     *   Explicar o que é o AWS Entity Resolution e como ele funciona.
>     *   Benefícios de unificar dados de diferentes fontes.
>     *   Casos de uso: CRM, marketing, análise de dados, etc.
>     *   Disponibilidade em novas regiões e o que isso significa para os clientes locais.
>     *   Incluir um exemplo de como o Entity Resolution pode identificar um cliente único em diferentes sistemas.
>     *   Call to action: "Melhore a qualidade dos seus dados com o AWS Entity Resolution!"
> 
> **8. Novidades de Segurança e Governança:**
> 
> *   **Pontos Relevantes:**
>     *   IAM Access Analyzer com política gerada automaticamente para S3.
>     *   Centralized Logging na Control Tower.
>     *   Suporte ao Verified Access no CloudTrail Lake.
>     *   Melhorias no Config Rules.
> 
> *   **Plano de Post:**
>     *   Título: "AWS Reforça a Segurança e Governança na Nuvem!"
>     *   Resumo das principais novidades em segurança e governança.
>     *   Detalhar cada uma das atualizações:
>         *   IAM Access Analyzer: Facilidade na criação de políticas S3.
>         *   Centralized Logging: Simplificação do registro de logs na Control Tower.
>         *   Verified Access no CloudTrail Lake: Visibilidade e controle de acesso a aplicações.
>         *   Config Rules: Melhorias na configuração de regras de conformidade.
>     *   Benefícios para a segurança e conformidade das aplicações AWS.
>     *   Incluir um gráfico mostrando a arquitetura de segurança da AWS.
>     *   Call to action: "Mantenha sua infraestrutura AWS segura e em conformidade com as últimas atualizações!"
> 
> **Escolha do Tópico Mais Relevante:**
> 
> Considerando o impacto geral e o interesse potencial do público, o tópico mais relevante para um post seria:
> 
> **Investimento Massivo no Chile.**
> 
> *   **Justificativa:**
>     *   O investimento de US$4 bilhões é uma notícia de grande impacto, mostrando o compromisso da AWS com a América Latina.
>     *   A construção de data centers gera empregos e desenvolvimento tecnológico na região.
>     *   A expansão da infraestrutura de nuvem beneficia empresas locais e internacionais.
>     *   A notícia tem apelo para um público amplo, incluindo empresas, desenvolvedores, investidores e interessados em tecnologia.
> 
> **Plano Detalhado para o Post Sobre o Investimento no Chile:**
> 
> *   **Título:** "AWS Investe US$4 Bilhões no Chile: Um Marco para a Nuvem na América Latina!"
> *   **Assuntos a Serem Abordados:**
>     1.  **Introdução:**
>         *   Apresentar o anúncio do investimento de US$4 bilhões da AWS no Chile.
>         *   Destacar a importância desse investimento para a região da América Latina.
>     2.  **Detalhes do Investimento:**
>         *   Número de data centers a serem construídos (três em Santiago).
>         *   Prazo do investimento (15 anos).
>         *   Número de zonas de disponibilidade (3).
>     3.  **Benefícios para a América Latina:**
>         *   Melhoria da infraestrutura de nuvem na região.
>         *   Redução da latência para aplicações e serviços.
>         *   Acesso a tecnologias avançadas como IA e machine learning.
>     4.  **Impacto Econômico e Social:**
>         *   Geração de empregos diretos e indiretos.
>         *   Desenvolvimento de habilidades digitais na região.
>         *   Contribuição para o crescimento econômico do Chile e da América Latina.
>     5.  **Sustentabilidade:**
>         *   Como os data centers serão resfriados (predominantemente por ar).
>         *   Compromisso da AWS com a sustentabilidade e a meta de zerar emissões líquidas de carbono até 2040.
>     6.  **Declarações de Executivos:**
>         *   Incluir citações de executivos da AWS sobre o investimento e seus benefícios.
>     7.  **Call to Action:**
>         *   Incentivar empresas e desenvolvedores a explorar os serviços da AWS na América Latina.
>         *   Oferecer recursos adicionais para saber mais sobre o investimento e as oportunidades.
> *   **Elementos Visuais:**
>     *   Imagem de um mapa da América Latina destacando o Chile.
>     *   Visualização de um data center da AWS.
>     *   Gráfico mostrando o crescimento da infraestrutura de nuvem na região.
> *   **Hashtags:**
>     *   \#AWS \#CloudComputing \#AmericaLatina \#Chile \#DataCenter \#Investimento \#Tecnologia \#Inovação
> 
> Este plano visa criar um post informativo, interessante e relevante para o público-alvo, destacando os principais benefícios do investimento da AWS no Chile e seu impacto na região.
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Redator) ---



> ## AWS turbina a nuvem na América Latina com investimento de 4 bilhões de dólares no Chile! 🚀🇨🇱
> 
> Preparem-se! A AWS acaba de anunciar um aporte massivo de **US$4 bilhões** para impulsionar a infraestrutura de nuvem no Chile, com a construção de **3 novos data centers** de última geração em Santiago. 🤯
> 
> Essa super novidade não só demonstra o compromisso da AWS com a nossa região, como também trará inúmeros benefícios para empresas e desenvolvedores:
> 
> *   **Menor latência:** Diga adeus aos atrasos!
> *   **Mais inovação:** Acesso facilitado a tecnologias de ponta, como IA e Machine Learning.
> *   **Geração de empregos:** Oportunidades para profissionais de tecnologia na América Latina.
> *   **Crescimento econômico:** Impulso para o desenvolvimento do Chile e da região.
> 
> E o melhor de tudo: os data centers serão construídos com foco na **sustentabilidade**, utilizando tecnologias de resfriamento eficientes e alinhadas com o compromisso da AWS de zerar as emissões de carbono até 2040. ♻️
> 
> Quer saber como a sua empresa pode se beneficiar dessa mega infraestrutura? 🤔
> 
> Acesse o link na bio e descubra um universo de possibilidades! 😉
> 
> \#AWS \#CloudComputing \#AmericaLatina \#Inovação
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Revisor) ---



> O rascunho está ótimo e pronto para publicar!


--------------------------------------------------------------
